## CycleGAN

* [CycleGAN](https://arxiv.org/pdf/1703.10593.pdf) with Kaggle Data, Painter by Numbers (https://www.kaggle.com/c/painter-by-numbers/data)
* First of all, you should import [GAN.py](https://github.com/tensorpack/tensorpack/blob/master/examples/GAN/GAN.py) at the same path.
* [Reference](https://github.com/tensorpack/tensorpack/blob/master/examples/GAN/CycleGAN.py) 

In [ ]:
import os
import argparse
import glob
from six.moves import range


from tensorpack import *
from tensorpack.dataflow import *
from tensorpack.callbacks import *
from tensorpack.tfutils import summary
from tensorpack.tfutils.summary import add_moving_summary
from tensorpack.utils.viz import stack_patches
from tensorpack.tfutils.scope_utils import auto_reuse_variable_scope
import tensorflow as tf
from GAN import *




import matplotlib.pyplot as plt

from GAN import *
from PIL import Image
from tensorpack import *
from tensorpack.dataflow import *
from tensorpack.callbacks import *
from tensorpack.tfutils import summary
from tensorpack.utils.viz import stack_patches
from tensorpack.tfutils.scope_utils import auto_reuse_variable_scope

## Dataflow

In [ ]:
class Gallery(RNGDataFlow):
    def __init__(self, painting_path, picture_path, shuffle=True):
        super(Gallery, self).__init__()
        self.painting_path = painting_path
        self.picture_path = picture_path
        self.shuffle = shuffle
                    
        """Read Data.
        Args:
            [img_path]: path for the images
        """
        paintings = []
        pictures = []
        min_width = 50000
        min_height = 60000
        
        # Read paintings
        for file in glob.iglob(self.painting_path + '/*.jpg'):
            img = Image.open(file).convert("RGB")
            img = np.array(img)
            paintings.append(img)
            print(img.shape)
            # Height
            if img.shape[1] <= min_height:
                min_height = img.shape[1]
            # Width
            if img.shape[0] <= min_width:
                min_width = img.shape[1]
                
            self.len = len(paintings)
            self.min_height = min_height
            self.min_width = min_width
        
        self.paintings = paintings
        
        # Read pictures
        for file in glob.iglob(self.picture_path + '/*.jpg'):
            img = Image.open(file).convert("RGB")
            img = np.array(img)
            pictures.append(img)
            
        self.pictures = pictures
            
    
    def __len__(self):
        return self.len
    
    
    def __iter__(self):
        idxs = np.arange(self.len)
        if self.shuffle:
            self.rng.shuffle(idxs)
        for k in idxs:
            yield [self.paintings[k], self.pictures[k]]
            
    def get_data(self):
        return self.__iter__()

## Image Handling

In [ ]:
import cv2
import glob

paintings_path = '/Users/chaeujeong/Desktop/Todays_painter/Data/Eyvind_Earle'
pictures_path = '/Users/chaeujeong/CycleGAN/datasets/cezanne2photo/trainB'

images = [cv2.imread(file) for file in glob.glob('path/to/files/*jpg')]








In [ ]:
def get_dataflow(batch_size):
    df = Gallery('./Paul_Cezanne/', './trainB', shuffle=True)

    # ----- Image Augmentation Options -------- #
    augs = [
        imgaug.CenterCrop((df.min_width, df.min_height)),
        imgaug.Resize((256, 256))
    ]
    df = AugmentImageComponent(df, augs)
    df = BatchData(df, batch_size)
    # start 5 processes to run the dataflow in parallel
#     df = PrefetchDataZMQ(df, 5)
    return df

In [ ]:
df = get_dataflow(4)
df.reset_state()

In [ ]:
fig =plt.figure(figsize=(256, 256))

for idx, dp in enumerate(df.get_data()):
    if idx == 0:
        for i in range(4):
            img = dp[idx][i]
            fig.add_subplot(1, 4, i + 1)
            plt.imshow(img)
        plt.show()
    break